### This notebook uses a breathing actuation sequence that computes triggers inflation deflation based upon an average inhale/exhale duration and a scale factor

In [1]:
from pythonosc import dispatcher, osc_server
from pythonosc.udp_client import SimpleUDPClient
import biofeatures
import threading
import numpy as np
import time
from random import uniform

In [2]:
def data_handler(unused_addr, args, data1, data2, data3, data4, data5, data6): #BITalino ServerBIT format
    global ecg_data, resp_data
    global osc_client
        
    B = args[0]
    HR = args[1]
    
    # BITalino ServerBIT
    ecg_data.append(float(np.random.rand(1)))
    resp_data.append(float(data6))
        
    if B.is_warmed_up:
        B.set_data(resp_data[-B.buffer_length:])
    
    if HR.is_warmed_up:
        HR.set_data(ecg_data[-HR.buffer_length:])


In [3]:
def riot_data_handler(unused_addr, args, *values): #R-IoT 
    global ecg_data, resp_data
    global osc_client
    
    B = args[0]
    HR = args[1]
    
#     ecg_data.append(float(values[13]))
    ecg_data.append(np.random.rand(1)[0])
    resp_data.append(float(values[13])) #TODO: CHANGE BACK
        
    if B.is_warmed_up:
        B.set_data(resp_data[-B.buffer_length:])
    
    if HR.is_warmed_up:
        HR.set_data(ecg_data[-HR.buffer_length:])
    


In [4]:
def warmup(B, resp_data, HR, ecg_data):
    """Function to launch once the system has warmed up.
    Sets the  data initially and launches a recursive update of features.
    Parameters
    ----------
    B: object containing breathing data and features
    resp_data: respiration signal data to set
    HR: object containing ECG data and features
    ecg_data: ecg signal data to set
    """
    
    if not B.is_warmed_up:
        print("Breathing WARMUP")
        B.set_data(resp_data[-B.buffer_length:])
        B.is_warmed_up = True
        B.update_loop()
        
    if not HR.is_warmed_up:
        print("ECG WARMUP")
        HR.set_data(ecg_data[-HR.buffer_length:])
        HR.is_warmed_up = True
        HR.update_loop()
        
#         act_launcher(B, resp_data, HR, ecg_data) # No thread based

        timer_act = threading.Timer(0.1, act_launcher, [B, resp_data, HR, ecg_data] ) # Thread based
        timer_act.start() # Thread based
        
#     breathing_factor = B.breathing_factor
    
#     if B.routine == "None":
#         print("n-----")

#     elif B.routine == "breathing_biofeedback":
#         timer_actuation1 = threading.Timer(0.1, breathing_biofeedback, ["1", B, True] )
#         timer_actuation1.start()
    
#         timer_actuation2 = threading.Timer(0.1, breathing_biofeedback, ["2", B, True] )
#         #timer_actuation2 = threading.Timer(0.1, pulsating, ["2", True] )
#         timer_actuation2.start()

#     elif B.routine == "pulsating":
#         pass

In [5]:
def act_launcher(B, resp_data, HR, ecg_data):
    global actuation_flag
    global act_one, act_two
    
    if B.routine == "None":
        print("n-------")

    elif B.routine == "breathing_biofeedback":
        breathing_factor = B.breathing_factor
        if act_one:
            timer_actuation1 = threading.Timer(0.1, breathing_biofeedback, ["1", B, True] )
            timer_actuation1.start()
        
        if act_two:
            timer_actuation2 = threading.Timer(0.1, breathing_biofeedback, ["2", B, True] )
            #timer_actuation2 = threading.Timer(0.1, pulsating, ["2", True] )
            timer_actuation2.start()

    elif B.routine == "pulsating":
        if act_one:
            timer_actuation1 = threading.Timer(0.1, pulsating, ["1", True] )
            timer_actuation1.start()

        if act_two:
            timer_actuation2 = threading.Timer(0.1, pulsating, ["2", True] )
            timer_actuation2.start()        

    elif B.routine == "random_inflation":
        if act_one:
            timer_actuation1 = threading.Timer(0.1, random_inflation , ["1", 2, 5, True] )
            timer_actuation1.start()
            
        if act_two:
            timer_actuation2 = threading.Timer(0.1, random_inflation, ["1", 2, 5, True] )
            timer_actuation2.start()        
    
    elif B.routine == "async_intervals":
        timer_actuation = threading.Timer(0.1, async_intervals, ["1", "2", 3, 3, True] )
        timer_actuation.start()

    elif B.routine == "async_breathing":
        timer_actuation = threading.Timer(0.1, async_breathing, ["1", "2", B, True] )
        timer_actuation.start()    
    
    elif B.routine.startswith("Deflate"):
        def_duration = int(B.routine[7:])
        
        if act_one:
            timer_actuation1 = threading.Timer(0.1, deflate, ["1", def_duration] )
            timer_actuation1.start()
        
        if act_two:
            timer_actuation1 = threading.Timer(0.1, deflate, ["2", def_duration] )
            timer_actuation1.start()
        
    elif B.routine.startswith("Inflate"):
        inf_duration = int(B.routine[7:])
        
        if act_one:
            timer_actuation1 = threading.Timer(0.1, warmup_inflate, ["1", inf_duration] )
            timer_actuation1.start()
        
        if act_two:
            timer_actuation1 = threading.Timer(0.1, warmup_inflate, ["2", inf_duration] )
            timer_actuation1.start()
        
    elif B.routine.startswith("Offset"):
        values = list(map(lambda x: int(x), B.routine.split(",")[1:]))
        offset_intervals("1", "2", values[0], values[1], values[2], B)

In [6]:
def warmup_inflate(act_id, duration):
    global osc_client
    
    print(duration)
    osc_client.send_message("/actuator/" + act_id + "/inflate", 100.0)
    time.sleep(duration)
    osc_client.send_message("/actuator/" + act_id + "/inflate", 0.0)


In [7]:
def deflate(act_id, duration):
    global osc_client
    
    osc_client.send_message("/actuator/" + act_id + "/inflate", -100.0)
    time.sleep(duration)
    osc_client.send_message("/actuator/" + act_id + "/inflate", 0.0)


### Different actuation sequences

In [8]:
# computes inhale and exhale interval duration as average duration of breath * breathing_factor 
def breathing_biofeedback(act_id, B, inflate):
    global osc_client 
    global actuation_flag
    global act_one, act_two
    
    if (act_id == "1" and not act_one) or (act_id == "2" and not act_two):
        osc_client.send_message("/actuator/" + act_id + "/inflate", 0.0)
        return
    
    if not actuation_flag or B.routine != "breathing_biofeedback":
        return


#     if not actuation_flag:
#         return
    
    breathing_factor = B.breathing_factor
#     print("BrF: ", breathing_factor)  
    
    if inflate:
        #print("inhale: ", B.features['avg_inhale'])
        osc_client.send_message("/actuator/" + act_id + "/inflate", 100.0)
        timer_exhale = threading.Timer(B.features['avg_inhale'] * breathing_factor, breathing_biofeedback, [act_id, B, False])
        timer_exhale.start()
        
        for i in range(1,10):
            osc_client.send_message("/actuator/" + act_id + "/inflate", 100.0)

    else:
        #print("exhale: ", B.features['avg_inhale'])
        osc_client.send_message("/actuator/" + act_id + "/inflate", -100.0)
        timer_inhale = threading.Timer(B.features['avg_inhale'] * breathing_factor, breathing_biofeedback, [act_id, B, True])
        timer_inhale.start()
        
        for i in range(1,10):
            osc_client.send_message("/actuator/" + act_id + "/inflate", -100.0)


In [19]:
# specify inhale and exhale interval duration in seconds
def intervals(act_id, inhale_duration, exhale_duration, inflate):
    global osc_client, actuation_flag
    global act_one, act_two
    
    if (act_id == "1" and not act_one) or (act_id == "2" and not act_two):
        osc_client.send_message("/actuator/" + act_id + "/inflate", 0.0)
        return
    
    if not actuation_flag or not B.routine.startswith("Offset"):
        return
    
    if inflate:
        print("inhale: ", inhale_duration)
        osc_client.send_message("/actuator/" + act_id + "/inflate", 100.0)
        timer_exhale = threading.Timer(inhale_duration, intervals, [act_id, inhale_duration, exhale_duration, False])
        timer_exhale.start()

    else:
        print("exhale: ", exhale_duration)
        osc_client.send_message("/actuator/" + act_id + "/inflate", -100.0)
        timer_inhale = threading.Timer(exhale_duration, intervals, [act_id, inhale_duration, exhale_duration, True])
        timer_inhale.start()

In [10]:
def pulsating(act_id,inflate):
    global osc_client
    global actuation_flag
    global act_one, act_two
    
    if (act_id == "1" and not act_one) or (act_id == "2" and not act_two):
        osc_client.send_message("/actuator/" + act_id + "/inflate", 0.0)
        return
    
    if not actuation_flag or B.routine != "pulsating":
        return
    
    if inflate:
        osc_client.send_message("/actuator/" + act_id + "/inflate", 50.0)
        timer_deflate = threading.Timer(3, pulsating, [act_id, False])
        timer_deflate.start()

    else:
        osc_client.send_message("/actuator/" + act_id + "/inflate", -50.0)
        timer_inflate = threading.Timer(3, pulsating, [act_id, True])
        timer_inflate.start()

In [11]:
def random_inflation(act_id, lower, upper, inflate):
    global osc_client, actuation_flag
    global act_one, act_two
    
    if (act_id == "1" and not act_one) or (act_id == "2" and not act_two):
        osc_client.send_message("/actuator/" + act_id + "/inflate", 0.0)
        return
    
    if not actuation_flag or B.routine != "random_inflation":        
        return
    
    int_length = uniform(lower,upper)
    
    if inflate:
        osc_client.send_message("/actuator/" + act_id + "/inflate", 100.0)
        timer_deflate = threading.Timer(int_length, random_inflation, [act_id, lower, upper, False])
        timer_deflate.start()

    else:
        osc_client.send_message("/actuator/" + act_id + "/inflate", -100.0)
        timer_inflate = threading.Timer(int_length, random_inflation, [act_id, lower, upper, True])
        timer_inflate.start()

In [12]:
def async_intervals(act_first, act_snd, inhale_dur, exhale_dur, inflate_first):
    global osc_client, actuation_flag
    
    if not actuation_flag or B.routine != "async_intervals":
        return
    
    if inflate_first:
        print("inhale: ", inhale_dur)
        osc_client.send_message("/actuator/" + act_first + "/inflate", 100.0)
        osc_client.send_message("/actuator/" + act_snd + "/inflate", -100.0)
        timer_inhale = threading.Timer(inhale_dur, async_intervals, [act_first, act_snd, inhale_dur, exhale_dur, False])
        timer_inhale.start()

    else:
        print("exhale: ", exhale_dur)
        osc_client.send_message("/actuator/" + act_first + "/inflate", -100.0)
        osc_client.send_message("/actuator/" + act_snd + "/inflate", 100.0)
        timer_exhale = threading.Timer(exhale_dur, async_intervals, [act_first, act_snd, inhale_dur, exhale_dur, True])
        timer_exhale.start()

In [13]:
def async_breathing(act_first, act_snd, B, inflate):
    global osc_client, actuation_flag
    
    if not actuation_flag or B.routine != "async_breathing":
        return

    breathing_factor = B.breathing_factor
    
    if inflate:
        print("inhale: ", B.features['avg_inhale'])
        osc_client.send_message("/actuator/" + act_first + "/inflate", 100.0)
        osc_client.send_message("/actuator/" + act_snd + "/inflate", -100.0)
        timer_exhale = threading.Timer(B.features['avg_inhale'] * breathing_factor, async_breathing, [act_first, act_snd, B, False])
        timer_exhale.start()

    else:
        print("exhale: ", B.features['avg_exhale'])
        osc_client.send_message("/actuator/" + act_first + "/inflate", -100.0)
        osc_client.send_message("/actuator/" + act_snd + "/inflate", 100.0)
        timer_inhale = threading.Timer(B.features['avg_exhale'] * breathing_factor, async_breathing, [act_first, act_snd, B, True])
        timer_inhale.start()

In [14]:
def offset_intervals(act_first, act_second, int_first, int_snd, offset, B):
    global actuation_flag
    
    if not actuation_flag or not B.routine.startswith("Offset"):
        return
    
    intervals(act_first, int_first, int_first, True);
    
    timer_snd = threading.Timer(offset, intervals, [act_second, int_snd, int_snd, True])
    timer_snd.start()


In [15]:
# computes degree of inflation/deflation based on ECG features

# TODO: how to compute the inflation/deflation value?

def heartrate_actuation(act_id, HR, timer_interval):
    global osc_client, actuation_flag
    global act_one, act_two
    
    if (act_id == "1" and not act_one) or (act_id == "2" and not act_two):
        osc_client.send_message("/actuator/" + act_id + "/inflate", 0.0)
        return
    
    if not actuation_flag:
        return
    
    current_trend = HR.current_trends["hr_mean"]
    #current_trend = HR.current_trends["rmssd"]
    #current_trend = HR.current_trends["LF/HF ratio"]
    
    current_feature = HR.features[-1]["hr_mean"]
    #current_feature = HR.features[-1]["rmssd"]
    #current_feature = HR.features[-1]["LF/HF ratio"]
    
    if current_trend > 0:
        osc_client.send_message("/actuator/" + act_id + "/inflate", 100.0)
        #osc_client.send_message("/actuator/" + act_id + "/inflate", current_trend * 1000/current_feature)
        timer_exhale = threading.Timer(timer_interval, heartrate_actuation, [act_id, HR, timer_interval])
        timer_exhale.start()

    else:
        #osc_client.send_message("/actuator/inflate", current_trend * 1000/current_feature)
        osc_client.send_message("/actuator/" + act_id + "/inflate", -100.0)
        timer_inhale = threading.Timer(timer_interval, heartrate_actuation, [act_id, HR, timer_interval])
        timer_inhale.start()

#### Set up configuration: IP (BITalino, R-IoT or Node-RED), port and OSC address pattern

In [16]:
def hold_down():
    global actuation_flag
    a = 0
    return

In [17]:
def conf_handler(unused_addr, args, *values): #Conf 

    global ecg_data, resp_data
    global actuation_flag
    global osc_client
    global act_one, act_two
    
    B = args[0]
    HR = args[1]   

    print("----------------- ACTUATION CHANGE ------------------------")
    print(values)
    
    if type(values[0]) == str and values[0].startswith("act"):
        if values[0] == "act1":
            if act_two:
                osc_client.send_message("/actuator/2/inflate", 0)
            act_one = True
            act_two = False
        elif values[0] == "act2":
            if act_one:
                osc_client.send_message("/actuator/1/inflate", 0)
            act_one = False
            act_two = True
        elif values[0] == "act12":
            act_one = True
            act_two = True
        
        timer_act = threading.Timer(0.1, act_launcher, [B, resp_data, HR, ecg_data] )
        #timer_act.start()
        
    elif type(values[0]) == str:
        if B.routine != values[0]:
            B.routine = values[0]

        if B.routine == "stop_all":
            actuation_flag = False
            val = 0.0
            osc_client.send_message("/actuator/1/inflate", val)
            osc_client.send_message("/actuator/2/inflate", val)
        
        if B.routine == "None":
            if act_one:
                act_one = False
                osc_client.send_message("/actuator/1/inflate", 0)
            if act_two:
                act_two = False
                osc_client.send_message("/actuator/2/inflate", 0)

        else:
            actuation_flag = True
                
            print(B.routine)
            timer_act = threading.Timer(3, act_launcher, [B, resp_data, HR, ecg_data] )
            timer_act.start()
        
    elif type(values[0]) == float: 
        B.breathing_factor = np.round(values[0],1)
        print("BrF val:", B.breathing_factor)
    
    else:
        print("NodeRED value: ", values[0])

In [20]:
# Definitions
bitalino_ip = '192.168.0.100'
bitalino_port = 8000
bitalino_srate = 100

riot_ip = "192.168.0.100"
riot_port = 31000 
riot_srate = 200

# riot_ip = "192.168.1.40"
# riot_port = 12000 

#actuator_ip = '192.168.0.110'
actuator_ip = '192.168.0.100'
actuator_port = 32000

riot_address = "/1/raw" #TODO: Change
bitalino_address = "/0/bitalino"


###### CONFIGURATION 
config_address = riot_address
config_ip = riot_ip
config_port = riot_port
srate = riot_srate 
C = {"config_address": config_address, "config_ip": config_ip, "config_port": config_port, "srate": srate}

osc_client = SimpleUDPClient(actuator_ip, actuator_port) 

actuation_flag = True

ecg_data = []
resp_data = []

act_one = True
act_two = True

B = biofeatures.breathing(data = np.ones(10), buffer_length=2000, srate=srate)
HRV = biofeatures.hrv(data = np.ones(10), buffer_length=2000, srate=srate)
B.breathing_factor = 1.0
B.routine = "breathing_biofeedback" 


my_dispatcher = dispatcher.Dispatcher()
my_dispatcher.map(config_address, riot_data_handler, B, HRV)
my_dispatcher.map("/brf/", conf_handler, B, HRV)


server = osc_server.ThreadingOSCUDPServer((config_ip, config_port), my_dispatcher)
print (C)
print("Serving on {}".format(server.server_address))

# Warmup
warmup_t = 10
timer_warmup = threading.Timer(warmup_t, warmup, [B,resp_data,HRV,ecg_data])
timer_warmup.start()

try: 
    server.serve_forever()
except KeyboardInterrupt:
    B.update_data_flag = False
    HRV.update_data_flag = False
    actuation_flag = False
    server.server_close()
    osc_client.send_message("/actuator/1/inflate", 0.0)
    osc_client.send_message("/actuator/2/inflate", 0.0)
except:
    raise

{'config_address': '/1/raw', 'config_ip': '192.168.0.100', 'config_port': 31000, 'srate': 200}
Serving on ('192.168.0.100', 31000)
Breathing WARMUP
ECG WARMUP
----------------- ACTUATION CHANGE ------------------------
('stop_all',)
stop_all
RESP ERROR


Exception in thread Thread-139285:
Traceback (most recent call last):
  File "/Users/annkatrin/opt/anaconda3/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/Users/annkatrin/opt/anaconda3/lib/python3.7/threading.py", line 1178, in run
    self.function(*self.args, **self.kwargs)
  File "/Users/annkatrin/Documents/Uni/Master/KTH/Master Project/biofeatures/notebooks/biofeatures.py", line 58, in update_loop
    self.resp_intervals(data)
  File "/Users/annkatrin/Documents/Uni/Master/KTH/Master Project/biofeatures/notebooks/biofeatures.py", line 87, in resp_intervals
    processed_data = resp.resp(signal=data, sampling_rate=self.srate, show=False)
  File "/Users/annkatrin/opt/anaconda3/lib/python3.7/site-packages/biosppy/signals/resp.py", line 91, in resp
    mirror=True)
  File "/Users/annkatrin/opt/anaconda3/lib/python3.7/site-packages/biosppy/signals/tools.py", line 609, in smoother
    (signal[0] * np.ones(size), signal, signal[-1] * np.ones(size)))
Ind

----------------- ACTUATION CHANGE ------------------------
('Offset,3,3,3',)
Offset,3,3,3
inhale:  3
exhale:  3
inhale:  3
inhale:  3
exhale:  3
exhale:  3
inhale:  3
----------------- ACTUATION CHANGE ------------------------
('stop_all',)
stop_all
RESP ERROR


Exception in thread Thread-145289:
Traceback (most recent call last):
  File "/Users/annkatrin/opt/anaconda3/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/Users/annkatrin/opt/anaconda3/lib/python3.7/threading.py", line 1178, in run
    self.function(*self.args, **self.kwargs)
  File "/Users/annkatrin/Documents/Uni/Master/KTH/Master Project/biofeatures/notebooks/biofeatures.py", line 58, in update_loop
    self.resp_intervals(data)
  File "/Users/annkatrin/Documents/Uni/Master/KTH/Master Project/biofeatures/notebooks/biofeatures.py", line 87, in resp_intervals
    processed_data = resp.resp(signal=data, sampling_rate=self.srate, show=False)
  File "/Users/annkatrin/opt/anaconda3/lib/python3.7/site-packages/biosppy/signals/resp.py", line 91, in resp
    mirror=True)
  File "/Users/annkatrin/opt/anaconda3/lib/python3.7/site-packages/biosppy/signals/tools.py", line 609, in smoother
    (signal[0] * np.ones(size), signal, signal[-1] * np.ones(size)))
Ind

RESP ERROR


Exception in thread Thread-145417:
Traceback (most recent call last):
  File "/Users/annkatrin/opt/anaconda3/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/Users/annkatrin/opt/anaconda3/lib/python3.7/threading.py", line 1178, in run
    self.function(*self.args, **self.kwargs)
  File "/Users/annkatrin/Documents/Uni/Master/KTH/Master Project/biofeatures/notebooks/biofeatures.py", line 58, in update_loop
    self.resp_intervals(data)
  File "/Users/annkatrin/Documents/Uni/Master/KTH/Master Project/biofeatures/notebooks/biofeatures.py", line 87, in resp_intervals
    processed_data = resp.resp(signal=data, sampling_rate=self.srate, show=False)
  File "/Users/annkatrin/opt/anaconda3/lib/python3.7/site-packages/biosppy/signals/resp.py", line 91, in resp
    mirror=True)
  File "/Users/annkatrin/opt/anaconda3/lib/python3.7/site-packages/biosppy/signals/tools.py", line 609, in smoother
    (signal[0] * np.ones(size), signal, signal[-1] * np.ones(size)))
Ind

----------------- ACTUATION CHANGE ------------------------
('act1',)
----------------- ACTUATION CHANGE ------------------------
('breathing_biofeedback',)
breathing_biofeedback
----------------- ACTUATION CHANGE ------------------------
('act12',)
----------------- ACTUATION CHANGE ------------------------
('None',)
----------------- ACTUATION CHANGE ------------------------
('breathing_biofeedback',)
breathing_biofeedback
----------------- ACTUATION CHANGE ------------------------
('act2',)
----------------- ACTUATION CHANGE ------------------------
('act12',)
----------------- ACTUATION CHANGE ------------------------
('breathing_biofeedback',)
breathing_biofeedback
----------------- ACTUATION CHANGE ------------------------
('stop_all',)
stop_all
----------------- ACTUATION CHANGE ------------------------
('act2',)
----------------- ACTUATION CHANGE ------------------------
('Deflate5',)
Deflate5
----------------- ACTUATION CHANGE ------------------------
('act1',)
---------------

## Launch on 2 devices

In [ ]:
from pythonosc import dispatcher, osc_server
from pythonosc.udp_client import SimpleUDPClient
import biofeatures
import time

# Use actuator IP
# Local Processing IP 192.168.0.100 or localhost 127.0.0.1, Arduino IP
actuator_ip = '127.0.0.1'
actuator_port = 32000
osc_client = SimpleUDPClient(actuator_ip, actuator_port) 


In [ ]:
val = 0.0
osc_client.send_message("/actuator/1/inflate", val)
osc_client.send_message("/actuator/2/inflate", val)


In [ ]:
val = 100.0
osc_client.send_message("/actuator/1/inflate", val)
osc_client.send_message("/actuator/2/inflate", val)


In [ ]:
deflate("1", 1)

In [ ]:
warmup_inflate("1", 3)

In [ ]:
warmup_inflate("2", 3)

In [ ]:
actuation_flag = True

pulsating("1", True)

In [ ]:
actuation_flag = False

In [ ]:
osc_client.send_message("/actuator/1/inflate", 0.0)

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
plt.plot(np.linspace(0,len(resp_data)/200., len(resp_data) ), resp_data)

In [6]:
testvar = "Offset,3,3,3"

list(map(lambda x: int(x), testvar.split(",")[1:]))

[3, 3, 3]